In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import boto3
from urllib.parse import unquote
import time
import re
from datetime import datetime
from supabase import create_client, Client
import warnings
import os 
from io import BytesIO

## Funciones

In [ ]:

links_exportaciones = []
# URL de la página que deseas capturar
url = "https://www.one.gob.do/datos-y-estadisticas/"

# Realiza la solicitud HTTP
response = requests.get(url)
response.encoding = 'utf-8'

# Analiza el contenido HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Encuentra todos los elementos 'div' con la clase 'elem'
bases = soup.find_all('div', class_='elem')

for base in bases:
    # Encuentra el div con la clase 'collapse' y el id especificado
    collapse_div = base.find('div', class_='collapse', id='acc_44c50024-924d-46d5-97f4-40d726159875')
   
    if collapse_div:
        # Encuentra todos los enlaces 'a' dentro de ese div
        links = collapse_div.find_all('a')
       
        # Imprime todos los href de los enlaces encontrados
        for link in links:
            href = link.get('href')
            if href and (href.endswith('.xlsx') or href.endswith('.xls')):
                print(href)
                links_exportaciones.append(href)

# Encabezados HTTP personalizados
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
}

# Columnas que queremos extraer
columnas = ['Transacción','Año','Mes','Cod_Seccion', 'Desc_Seccion', 'Cod_Capitulo', 'Desc_Capitulo', 'Cod_Partida', 'Desc_Partida', 'Cod_Arancel', 'Desc_Arancel', 'Region', 'Pais_Destino', 'Via_Transporte', 'Regimen_Aduanero','Colecturia_ID', 'Colecturia', 'Peso', 'V_FOB_USD']
#'V_CIF_USD'
# Lista para almacenar los DataFrames
dataframes = []

# Lista para registrar errores
errores = []

# Descargar y procesar los archivos Excel
for link in links_exportaciones:
    for intento in range(3):  # Intentar hasta 3 veces
        try:
            print(f"Procesando: {link}")
            response = requests.get(link, headers=headers, timeout=30)
            response.raise_for_status()  # Verificar si hubo algún error en la solicitud

            # Leer el contenido del archivo Excel
            df = pd.read_excel(BytesIO(response.content), usecols=columnas)
            print(f"Archivo leído exitosamente: {link}")
            dataframes.append(df)
            break  # Salir del bucle si el archivo se descargó correctamente
        except requests.exceptions.RequestException as e:
            print(f"Error al procesar {link}, intento {intento + 1}: {e}")
            time.sleep(5)  # Esperar 5 segundos antes de reintentar
            if intento == 2:  # Si es el último intento, registrar el error
                errores.append(link)
        except Exception as e:
            print(f"Error inesperado al procesar {link}: {e}")
            errores.append(link)
            break

# Consolidar todos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)



In [ ]:
df_final.head()

In [ ]:
df_final.dtypes

In [11]:
# Crear una copia del DataFrame original
working_df = df_final.copy()

In [ ]:
working_df.dtypes

In [ ]:
# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================
# Este bloque establece columnas iniciales con valores estáticos y ordena el DataFrame.
# Se asume que 'working_df' está definido anteriormente.

# Añadir columnas iniciales con valores estáticos
working_df['Código ISO'] = 'DOM'
working_df['País'] = 'República Dominicana'
working_df['Departamento'] = 'Nacional'
working_df['Ciudad'] = 'Nacional'
working_df['Categoría'] = 'Sector Externo'
working_df['Sub-Categoría'] = 'Exportaciones'

# Ordenar las columnas en el DataFrame
column_order = [
    'Código ISO',  'País', 'Departamento', 'Ciudad', 'Categoría', 'Sub-Categoría',
    'Pais_Destino', 'Region', 'Via_Transporte', 'Colecturia', 'Regimen_Aduanero',
    'Cod_Seccion', 'Cod_Capitulo', 'Cod_Partida', 'Cod_Arancel',
    'Transacción', 'Año', 'Mes',   'Desc_Seccion', 'Desc_Capitulo',
    'Desc_Partida','Desc_Arancel', 'Peso', 'V_FOB_USD',
    'Colecturia_ID']
working_df = working_df[column_order]

# =============================================================================
# PREPROCESAMIENTO DE DATOS
# =============================================================================
# Se eliminan guiones "-" en las columnas descriptivas y se generan métricas base.
# Además, se crea una columna 'Fecha' a partir de Año/Mes y se elimina estas columnas.

# Preprocesar texto: reemplazar "-" por espacios y eliminar espacios extra
for col in ['Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel']:
    working_df[col] = working_df[col].str.replace('-', ' ').str.strip()

# Crear columnas derivadas a partir de las columnas base
working_df['Toneladas'] = working_df['Peso'] / 1000.0
working_df['Millones_USD_FOB'] = working_df['V_FOB_USD'] / 1_000_000
working_df['Millones_Toneladas_metricas'] = working_df['Toneladas'] / 1_000_000

# Crear columna 'Fecha' a partir de 'Año' y 'Mes' y luego eliminar dichas columnas
working_df['Fecha'] = pd.to_datetime(
    working_df['Año'].astype(str) + '-' + working_df['Mes'].apply(lambda x: f'{x:02}') + '-01'
)
working_df.drop(['Año', 'Mes'], axis=1, inplace=True)

# Ordenar el DataFrame por criterios lógicos (fecha y desagregaciones)
working_df.sort_values(by=['Fecha', 'Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel'],
                       ascending=True, inplace=True)

# =============================================================================
# AGRUPACIÓN MENSUAL Y CÁLCULOS DE SUMAS A 12 MESES
# =============================================================================
# Se agrupan los datos mensuales por desagregaciones y se obtienen sumas.
# Luego se calculan sumas móviles a 12 meses (12M) para las métricas clave.

# Agrupar mensualmente, agregando columnas con función 'first' o 'sum' según corresponda
agrupadas = working_df.groupby(
    ['Fecha', 'Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel',
     'Cod_Seccion', 'Cod_Capitulo', 'Cod_Partida', 'Cod_Arancel'],
    as_index=False
).agg({
    'Toneladas': 'sum',
    'Millones_USD_FOB': 'sum',
    'Millones_Toneladas_metricas': 'sum',
    'Pais_Destino': 'first',
    'Region': 'first',
    'Via_Transporte': 'first',
    'Colecturia_ID': 'first',
    'Colecturia': 'first',
    'Regimen_Aduanero': 'first'
})

# Extraer el mes para cálculos de rolling y variaciones
agrupadas['Mes'] = agrupadas['Fecha'].dt.month

# Calcular sumas móviles a 12 meses
grouped = agrupadas.groupby(['Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida'])
agrupadas['Toneladas_12M'] = grouped['Toneladas'].transform(lambda x: x.rolling(window=12, min_periods=1).sum())
agrupadas['Millones_USD_FOB_12M'] = grouped['Millones_USD_FOB'].transform(lambda x: x.rolling(window=12, min_periods=1).sum())
agrupadas['Millones_Toneladas_metricas_12M'] = grouped['Millones_Toneladas_metricas'].transform(lambda x: x.rolling(window=12, min_periods=1).sum())

# =============================================================================
# CÁLCULO DE VALOR ANTERIOR Y VARIACIONES ANUALES
# =============================================================================
# Se calcula el valor del mismo mes del año anterior para cada métrica y
# a partir de allí se obtiene la variación porcentual anual.

columnas_id = [
    'Fecha', 'Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel',
    'Cod_Seccion', 'Cod_Capitulo', 'Cod_Partida', 'Cod_Arancel', 'Mes',
    'Pais_Destino', 'Region', 'Via_Transporte', 'Colecturia_ID', 'Colecturia', 'Regimen_Aduanero'
]

columnas_con_valor = [c for c in agrupadas.columns if c not in columnas_id]

# Calcular valor anterior (mismo mes del año previo) y variaciones
for col in columnas_con_valor:
    agrupadas[f'{col}_valor_anterior'] = agrupadas.groupby('Mes')[col].shift(12)
    agrupadas[f'variacion {col}'] = ((agrupadas[col] - agrupadas[f'{col}_valor_anterior']) /
                                     agrupadas[f'{col}_valor_anterior'] * 100)

# Filtrar columnas para el melt (excluyendo las de valor_anterior)
final_columnas_con_valor = [col for col in agrupadas.columns if col not in columnas_id and 'valor_anterior' not in col.lower()]

# =============================================================================
# TRANSFORMACIÓN A FORMATO LARGO (MELT) EN CHUNKS
# =============================================================================
# Para manejar grandes volúmenes de datos sin problemas de memoria,
# se realiza el melt en trozos (chunks).

chunk_size = 100000  # Ajustar según memoria
chunks = [final_columnas_con_valor[i:i+chunk_size] for i in range(0, len(final_columnas_con_valor), chunk_size)]

df_long_list = []
for chunk in chunks:
    df_partial = pd.melt(
        agrupadas,
        id_vars=columnas_id,
        value_vars=chunk,
        value_name='Valor',
        var_name='Columna_desagregacion'
    )
    df_long_list.append(df_partial)
    del df_partial

df_long = pd.concat(df_long_list, ignore_index=True)
del df_long_list

# Añadir columnas adicionales con información fija
df_long = df_long.assign(
    Codigo_ISO='DOM',
    Categoria='Sector Externo',
    Frecuencia='Mensual',
    Fuente='ONE',
    Pais='República Dominicana',
    Departamento='Nacional',
    Ciudad='Nacional',
    Subcategoria='Importaciones'
)

# =============================================================================
# DEFINICIÓN DE FUNCIONES PARA ASIGNACIÓN DE UNIDAD, TIPO Y AJUSTE
# =============================================================================
# Estas funciones asignan unidades, tipo de dato y ajuste temporal
# según el nombre de la columna desagregada.

def columna_unidad(row):
    """
    Determina la unidad de medida a partir del nombre de la variable.
    
    Ajusta las palabras clave según las variables existentes en tu DataFrame.
    Si nunca obtienes '% de las Importaciones totales' o 'USD por tonelada',
    verifica los nombres de las columnas o remueve estas condiciones.
    """
    var = str(row['Columna_desagregacion']).lower()

    # Ejemplo: Si tus columnas de participación se llaman así:
    if 'porcentaje_importaciones_totales' in var or 'participacion_importaciones_totales' in var:
        return '% de las Importaciones totales'

    # Ejemplo: Si tus columnas de precio por tonelada se llaman así:
    elif 'usd_por_ton' in var or 'precio_implicito' in var:
        return 'USD por tonelada'

    elif 'variacion' in var:
        return '% variación anual'

    elif 'millones' in var:
        return 'Millones'

    # Si llegamos hasta aquí es porque el nombre de la variable
    # no coincidió con ninguna de las condiciones anteriores.
    return 'Unidad desconocida'


def columna_tipo(row):
    """
    Determina el tipo de dato (USD FOB, USD CIF, Toneladas métricas, etc.)
    en función de la variable.
    
    Parámetros:
        row (pd.Series): Fila del DataFrame, se usa 'Columna_desagregacion' para decidir el tipo.
    
    Retorno:
        str: Tipo correspondiente al valor.
    """
    var = str(row['Columna_desagregacion']).lower()
    if 'fob' in var:
        return 'USD FOB'
    elif 'toneladas' in var and 'variacion' not in var and 'precio_implicito' not in var:
        return 'Toneladas métricas'
    elif 'variacion' in var:
        return 'Precio implícito'
    else:
        return 'Tipo desconocido'

def columna_ajuste(row):
    """
    Determina el ajuste temporal de la serie (mensual o suma móvil 12 meses)
    según la variable.
    
    Parámetros:
        row (pd.Series): Fila del DataFrame, se usa 'Columna_desagregacion' para decidir el ajuste.
    
    Retorno:
        str: Ajuste temporal ('Suma móvil 12 meses' o 'Serie mensual').
    """
    var = str(row['Columna_desagregacion']).lower()
    if '12m' in var:
        return 'Suma móvil 12 meses'
    else:
        return 'Serie mensual'

# Aplicar las funciones de asignación
df_long['Unidad'] = df_long.apply(columna_unidad, axis=1)
df_long['Tipo'] = df_long.apply(columna_tipo, axis=1)
df_long['Ajuste'] = df_long.apply(columna_ajuste, axis=1)

# Renombrar columnas de desagregación para mayor claridad
df_long.rename(columns={
    'Desc_Arancel': 'Desagregacion-4',
    'Desc_Partida': 'Desagregacion-3',
    'Desc_Capitulo': 'Desagregacion-2',
    'Desc_Seccion': 'Desagregacion-1'
}, inplace=True)

# =============================================================================
# RESULTADO FINAL
# =============================================================================
# 'df_long' contiene los datos en formato largo, con columnas id, valor, unidad,
# tipo, ajuste y otras variables descriptivas.

print(df_long.head())




In [ ]:
df_long.sort_values(by=['Fecha', 'Desagregacion-1', 'Desagregacion-2', 'Desagregacion-3','Desagregacion-4'], ascending=True, inplace=True)
df_long

In [ ]:
df_long.dtypes

In [ ]:
# Diccionario de mapeo: 'columna_original': 'columna_nueva'
nuevos_nombres = {
    'Fecha': 'Fecha',
    'Desagregacion-1': 'Sección',
    'Desagregacion-2': 'Capítulo',
    'Desagregacion-3': 'Partida',
    'Desagregacion-4': 'Arancel',
    'Cod_Seccion': 'Código Sección',
    'Cod_Capitulo': 'Código Capítulo',
    'Cod_Partida': 'Código Partida',
    'Cod_Arancel': 'Código Arancel',
    'Pais_Origen': 'País Origen',
    'Region': 'Región',
    'Via_Transporte': 'Vía Transporte',
    'Colecturia_ID': 'Colecturía ID',
    'Colecturia': 'Colecturía',
    'Regimen_Aduanero': 'Régimen Aduanero',
    'Valor': 'Valor',
    'Codigo_ISO': 'Código ISO',
    'Categoria': 'Categoría',
    'Frecuencia': 'Frecuencia',
    'Pais': 'País',
    'Departamento': 'Departamento',
    'Subcategoria': 'Sub-Categoría',
    'Unidad': 'Unidad',
    'Tipo': 'Tipo',
    'Ajuste': 'Ajuste',
    'Fuente': 'Fuente'
}

# Aplicar el renombrado de columnas
df_long.rename(columns=nuevos_nombres, inplace=True)

# Verificar el resultado
print(df_long.head())


In [ ]:
df_long["Arancel"]

In [ ]:
len(df_long)

In [ ]:
df_long.duplicated().sum()

In [25]:
df_long_copy = df_long.copy()

In [26]:
df_long_copy.rename(columns={
            'Código ISO':'Codigo_ISO',
            'Sub-Categoría':'Sub-Categoria',
            'Categoría': 'Categoria',
            'País': 'Pais',
            'Pais_Destino': 'Pais_Origen',
            'Régimen Aduanero': 'Regimen_Aduanero',
            'Código Sección': 'Codigo_Seccion',
            'Sección': 'Seccion',
            'Código Arancel': 'Codigo_Arancel',
            'Código Capítulo': 'Codigo_Capitulo',
            'Código Partida': 'Codigo_Partida',
            'Capítulo': 'Capitulo',
            'Región': 'Region',
            'Vía Transporte': 'Via_Transporte',
            'Colecturía ID': 'Colecturia_ID',
            'Colecturía': 'Colecturia'
        }, inplace=True)

df_long_copy.drop(columns=['Mes', 'Columna_desagregacion'], inplace=True)

In [ ]:
df_long_copy.columns

In [28]:
df_long_copy.to_csv('outputs/data_final.csv')

In [ ]:
def opciones_unicas_con_conteo(df, columna):
    """
    Muestra todas las opciones únicas de una columna en un DataFrame y 
    cuántas veces aparece cada una.

    Args:
        df (pd.DataFrame): El DataFrame a analizar.
        columna (str): El nombre de la columna a verificar.

    Returns:
        pd.Series: Serie con los valores únicos y su conteo.
    """
    conteo = df[columna].value_counts(dropna=False)  # Incluye NaN si existen
    print(f"Opciones únicas en la columna '{columna}' y su conteo:")
    for valor, cantidad in conteo.items():
        print(f"- {valor}: {cantidad}")
    return conteo

# Ejemplo de uso
conteos = opciones_unicas_con_conteo(df_long, 'Columna_desagregacion')


In [20]:
chunk_size = 700000  # Ajustar según la memoria y el tamaño de df_long

output_file = 'df_long_output.json'

# Abrimos el archivo en modo escritura al iniciar
with open(output_file, 'w', encoding='utf-8') as f:
    # Iterar sobre el DataFrame en chunks
    for i in range(0, len(df_long), chunk_size):
        chunk_df = df_long.iloc[i:i+chunk_size]

        # Convertir el chunk a JSON line-by-line (orient='records', lines=True)
        chunk_json = chunk_df.to_json(orient='records', lines=True, force_ascii=False, date_format='iso', date_unit='s')

        # Escribir el chunk en el archivo
        f.write(chunk_json)

In [ ]:
def dividir_dataframe(df, nombre_base='parte'):
    """
    Divide un DataFrame en 10 partes iguales y guarda cada parte en un CSV.
   
    Parámetros:
    - df: DataFrame a dividir
    - nombre_base: prefijo para los nombres de los archivos CSV (por defecto 'parte')
    """
    # Calcula el número de filas por parte
    filas_por_parte = len(df) // 25
   
    # Asegura que se incluyan todas las filas, incluso si no se dividen perfectamente
    partes = [df[i:i+filas_por_parte] for i in range(0, len(df), filas_por_parte)]
   
    # Recorta a 10 partes si hay más
    partes = partes[:25]
   
    # Guarda cada parte en un CSV
    for i, parte in enumerate(partes, 1):
        nombre_archivo = f'{nombre_base}_{i}.csv'
        parte.to_csv('outputs/'+nombre_archivo, index=False)
        print(f'Guardado: {nombre_archivo} (filas: {len(parte)})')
   
dividir_dataframe(df_long,'parte')

In [ ]:
import pandas as pd
import os

# Definir el mapeo de nombres de columnas
column_mapping = {
    'Fecha': 'Fecha',
    'Sección': 'Seccion',
    'Capítulo': 'Capitulo',
    'Partida': 'Partida',
    'Arancel': 'Arancel',
    'Código Sección': 'Codigo_Seccion',
    'Código Capítulo': 'Codigo_Capitulo',
    'Código Partida': 'Codigo_Partida',
    'Código Arancel': 'Codigo_Arancel',
    'País Origen': 'Pais_Origen',
    'Región': 'Region',
    'Vía Transporte': 'Via_Transporte',
    'Colecturía ID': 'Colecturia_ID',
    'Colecturía': 'Colecturia',
    'Régimen Aduanero': 'Regimen_Aduanero',
    'Valor': 'Valor',
    'Código ISO': 'Codigo_ISO',
    'Categoría': 'Categoria',
    'Frecuencia': 'Frecuencia',
    'País': 'Pais',
    'Departamento': 'Departamento',
    'Sub-Categoria': 'Sub-Categoria',
    'Unidad': 'Unidad',
    'Tipo': 'Tipo',
    'Ajuste': 'Ajuste',
    'Fuente': 'Fuente'
}

# Leer y renombrar el archivo CSV
def rename_csv_columns(input_file, output_file):
    # Cargar el archivo CSV
    df = pd.read_csv(input_file)
    
    # Renombrar columnas
    df.rename(columns=column_mapping, inplace=True)
    
    # Guardar el CSV corregido
    df.to_csv(output_file, index=False)
    print(f"Archivo procesado y guardado como: {output_file}")

# Procesar archivos CSV en la carpeta actual
input_folder = "csv"
output_folder = "salida"

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre los archivos CSV
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        input_path = os.path.join(input_folder, file)
        output_path = os.path.join(output_folder, f"corregido_{file}")
        rename_csv_columns(input_path, output_path)


In [64]:
def process_data_in_chunks(datos, chunk_size, supabase):
    # Fetch existing dimensions from the dimension table
    def fetch_existing_dimensions():
        response = supabase.table('tabladimension').select('*').execute()
        
        # Si la respuesta no contiene datos, retornamos estructuras vacías
        if not response.data or len(response.data) == 0:
            return {}, []

        # Caso en que hay datos
        columns = response.data[0].keys()
        dimension_columns = [col for col in columns if col != 'id']

        existing_dimensions = {}
        for row in response.data:
            dimension_combination = tuple(row[col] for col in dimension_columns)
            existing_dimensions[dimension_combination] = row['id']

        return existing_dimensions, dimension_columns
    def get_or_add_dimension(dimension_combination):
        if dimension_combination in existing_dimensions:
            return existing_dimensions[dimension_combination]
        else:
            # Add new dimension to the dimension table
            new_dimension_data = dict(zip(dimension_columns, dimension_combination))
            print(new_dimension_data)
            response = supabase.table('tabladimension').insert(new_dimension_data).execute()
            new_id = response.data[0]['id']
            existing_dimensions[dimension_combination] = new_id
            return new_id
 
    def create_dimension_combination(row):
        return tuple(row[col] for col in dimension_columns)
   
    existing_dimensions, dimension_columns = fetch_existing_dimensions()
    errores = []  # To track errors
    total_chunks = (len(datos) + chunk_size - 1) // chunk_size
    # Define columns to keep in the final upsert
    columns_to_keep = ['Fecha', 'Valor', 'id']
   
    for i in range(total_chunks):
        start_index = i * chunk_size
        end_index = (i + 1) * chunk_size
        chunk_data = datos[start_index:end_index]
 
        # Prepare data for upsert
        for row in chunk_data:
            dimension_combination = create_dimension_combination(row)
            dimension_id = get_or_add_dimension(dimension_combination)
            row['id'] = dimension_id
 
        # Drop columns not in the columns_to_keep
        chunk_data_filtered = [{k: row[k] for k in columns_to_keep} for row in chunk_data]
 
        # Upsert data into the main table
        try:
            response = supabase.table('datos_light').upsert(chunk_data_filtered).execute()
            print(response)
        except Exception as e:
            errores.append(str(e))
 
    return f"Terminamos errores: {', '.join(errores)}"


In [65]:
from botocore.exceptions import ClientError
import json
def get_secret():
 
    secret_name = "RepDom-DB"
    region_name = "us-east-1"
 
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
 
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e
 
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret


secretos=get_secret()

url_supabase: str = secretos['supabase_URL_RD']
key: str = secretos['supabase_key_RD']

In [ ]:


df=pd.read_csv("salida\corregido_parte_1.csv") 
df = df.dropna(subset=['Fecha', 'Valor']).drop_duplicates()

# Convierte las columnas a cadena de texto si es necesario
for col in df.columns:
    if col != 'Valor':  # 'Valor' debe mantenerse en su tipo original
        df[col] = df[col].astype(str)

# Convierte a diccionario
datos = df.to_dict(orient='records')
supabase: Client = create_client(url_supabase, key)


In [103]:
test_data = datos[:10]



In [ ]:
process_data_in_chunks(test_data,10,supabase)

In [ ]:
# Extraer la primera fila del DataFrame
row = df.iloc[0].to_dict()
print("\nPrimera fila del DataFrame:", row)

# Función fetch_existing_dimensions (probamos si funciona correctamente)
def fetch_existing_dimensions():
    print("\n--- Ejecutando fetch_existing_dimensions ---") 
    response = supabase.table('tabladimension').select('*').execute()
    print("Response data:", response.data)

    if not response.data or len(response.data) == 0:
        # Si no hay datos en la tabla, definimos las columnas manualmente
        dimension_columns = ['Seccion', 'Capitulo', 'Partida', 'Arancel', 'Codigo_Seccion', 
                             'Codigo_Capitulo', 'Codigo_Partida', 'Codigo_Arancel', 
                             'Pais_Origen', 'Region', 'Via_Transporte', 'Colecturia_ID', 
                             'Colecturia', 'Regimen_Aduanero', 'Codigo_ISO', 'Categoria', 
                             'Frecuencia', 'Pais', 'Departamento', 'Sub-Categoria', 
                             'Unidad', 'Tipo', 'Ajuste', 'Fuente']
        return {}, dimension_columns

    columns = response.data[0].keys()
    dimension_columns = [col for col in columns if col != 'id']

    existing_dimensions = {}
    for r in response.data:
        dimension_combination = tuple(r[c] for c in dimension_columns)
        existing_dimensions[dimension_combination] = r['id']

    print("Existing dimensions:", existing_dimensions)
    print("Dimension columns:", dimension_columns)
    return existing_dimensions, dimension_columns

# Función create_dimension_combination
def create_dimension_combination(row):
    print("\n--- Ejecutando create_dimension_combination ---")
    combination = tuple(row[col] for col in dimension_columns)
    print("Dimension combination:", combination)
    return combination

# Función get_or_add_dimension
def get_or_add_dimension(dimension_combination):
    print("\n--- Ejecutando get_or_add_dimension ---")
    if dimension_combination in existing_dimensions:
        print("ID existente encontrado:", existing_dimensions[dimension_combination])
        return existing_dimensions[dimension_combination]
    else:
        new_dimension_data = dict(zip(dimension_columns, dimension_combination))
        print("Insertando nueva dimensión:", new_dimension_data)
        response = supabase.table('tabladimension').insert(new_dimension_data).execute()
        new_id = response.data[0]['id']
        print("Nuevo ID retornado:", new_id)
        existing_dimensions[dimension_combination] = new_id
        return new_id

# Proceso paso a paso con la primera fila
print("\n--- INICIANDO PRUEBA PASO A PASO ---")

# Paso 1: Fetch existing dimensions
existing_dimensions, dimension_columns = fetch_existing_dimensions()

# Paso 2: Crear la combinación de dimensiones
dimension_combination = create_dimension_combination(row)

# Paso 3: Obtener o agregar la dimensión
dimension_id = get_or_add_dimension(dimension_combination)

# Paso 4: Preparar datos para upsert
row['id'] = dimension_id
columns_to_keep = ['Fecha', 'Valor', 'id']
chunk_data_filtered = {k: row[k] for k in columns_to_keep}
print("\nDatos preparados para upsert:", chunk_data_filtered)

# Paso 5: Realizar el upsert en Supabase
try:
    print("\n--- Ejecutando upsert en datos_light ---")
    response = supabase.table('datos_light').upsert([chunk_data_filtered]).execute()
    print("Upsert response:", response.data)
except Exception as e:
    print("Error durante el upsert:", e)


In [ ]:
# Revisar y eliminar filas con valores NaN en las columnas Fecha o Valor
def clean_dataframe(df):
    print("--- Revisión de valores NaN ---")
    
    # Contar los NaN en las columnas relevantes
    print("Conteo de NaN antes de limpiar:")
    print(df[['Fecha', 'Valor']].isnull().sum())
    
    # Eliminar filas con NaN en las columnas 'Fecha' o 'Valor'
    df_cleaned = df.dropna(subset=['Fecha', 'Valor'])
    
    # Verificar nuevamente
    print("\nConteo de NaN después de limpiar:")
    print(df_cleaned[['Fecha', 'Valor']].isnull().sum())
    
    # Mostrar cuántas filas se eliminaron
    rows_removed = len(df) - len(df_cleaned)
    print(f"\nSe eliminaron {rows_removed} filas con NaN en 'Fecha' o 'Valor'.")
    
    return df_cleaned

# Aplicar la función al DataFrame
df = clean_dataframe(df)

# Opcional: Guardar el DataFrame limpio para futuras referencias



In [105]:

supabase: Client = create_client(url_supabase, key)

In [ ]:
import pandas as pd

# Cargar el DataFrame
def split_dataframe(df):
    """
    Divide el DataFrame en dos:
    1. Dimensiones fijas (tabladimension)
    2. Datos dinámicos (datos_light)
    """
    dimension_columns = [col for col in df.columns if col not in ['Fecha', 'Valor']]

    # Extraer dimensiones únicas
    dimensiones = df[dimension_columns].drop_duplicates().reset_index(drop=True)
    dimensiones['id'] = range(1, len(dimensiones) + 1)  # Agregar IDs únicos

    # Mapear IDs a los datos dinámicos
    datos = df.merge(dimensiones, on=dimension_columns, how='left')[['Fecha', 'Valor', 'id']]

    return dimensiones, datos

# Cargar archivo CSV
df = pd.read_csv('salida/corregido_parte_1.csv')
df = df.dropna(subset=['Fecha', 'Valor']).drop_duplicates()

# Dividir el DataFrame en dimensiones y datos
dimensiones, datos = split_dataframe(df)

# Guardar los DataFrames resultantes en archivos CSV
dimensiones.to_csv('prueba/dimensiones.csv', index=False)
datos.to_csv('prueba/datos.csv', index=False)

print("Dimensiones y datos divididos y guardados como CSV.")


In [ ]:
import pandas as pd
import os

# Cargar y dividir un DataFrame
def split_dataframe(df):
    """
    Divide el DataFrame en dos:
    1. Dimensiones fijas (tabladimension)
    2. Datos dinámicos (datos_light)
    """
    dimension_columns = [col for col in df.columns if col not in ['Fecha', 'Valor']]

    # Extraer dimensiones únicas
    dimensiones = df[dimension_columns].drop_duplicates().reset_index(drop=True)
    dimensiones['id'] = range(1, len(dimensiones) + 1)  # Agregar IDs únicos

    # Mapear IDs a los datos dinámicos
    datos = df.merge(dimensiones, on=dimension_columns, how='left')[['Fecha', 'Valor', 'id']]

    return dimensiones, datos

def process_multiple_csv(input_folder, output_folder):
    """
    Procesa todos los archivos CSV en una carpeta, dividiéndolos en dimensiones y datos.
    """
    os.makedirs(output_folder, exist_ok=True)
    archivos = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    all_dimensiones = []
    all_datos = []

    for archivo in archivos:
        print(f"Procesando: {archivo}")
        # Cargar archivo CSV
        df = pd.read_csv(os.path.join(input_folder, archivo))
        df = df.dropna(subset=['Fecha', 'Valor']).drop_duplicates()

        # Dividir el DataFrame
        dimensiones, datos = split_dataframe(df)

        # Agregar a las listas generales
        all_dimensiones.append(dimensiones)
        all_datos.append(datos)

    # Combinar y guardar dimensiones únicas
    combined_dimensiones = pd.concat(all_dimensiones).drop_duplicates().reset_index(drop=True)
    combined_dimensiones.to_csv(os.path.join(output_folder, 'dimensiones.csv'), index=False)

    # Combinar y guardar datos
    combined_datos = pd.concat(all_datos).reset_index(drop=True)
    combined_datos.to_csv(os.path.join(output_folder, 'datos.csv'), index=False)

    print("Todos los archivos han sido procesados y guardados.")

# Procesar todos los CSV en la carpeta 'entrada' y guardar en la carpeta 'salida'
process_multiple_csv('entrada', 'salida')


In [ ]:
import pandas as pd
import os

# Cargar y dividir un DataFrame
def split_dataframe(df):
    """
    Divide el DataFrame en dos:
    1. Dimensiones fijas (tabladimension)
    2. Datos dinámicos (datos_light)
    """
    dimension_columns = [col for col in df.columns if col not in ['Fecha', 'Valor']]

    # Extraer dimensiones únicas
    dimensiones = df[dimension_columns].drop_duplicates().reset_index(drop=True)
    dimensiones['id'] = range(1, len(dimensiones) + 1)  # Agregar IDs únicos

    # Mapear IDs a los datos dinámicos
    datos = df.merge(dimensiones, on=dimension_columns, how='left')[['Fecha', 'Valor', 'id']]

    return dimensiones, datos

def process_multiple_csv(input_folder, output_folder):
    """
    Procesa todos los archivos CSV en una carpeta, dividiéndolos en dimensiones y datos.
    """
    os.makedirs(output_folder, exist_ok=True)
    archivos = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    all_dimensiones = []
    all_datos = []

    for archivo in archivos:
        print(f"Procesando: {archivo}")
        # Cargar archivo CSV
        df = pd.read_csv(os.path.join(input_folder, archivo))

        # Eliminar filas con valores nulos en 'Fecha' o 'Valor'
        df = df.dropna(subset=['Fecha', 'Valor']).drop_duplicates()

        # Dividir el DataFrame
        dimensiones, datos = split_dataframe(df)

        # Agregar a las listas generales
        all_dimensiones.append(dimensiones)
        all_datos.append(datos)

    # Combinar y guardar dimensiones únicas
    combined_dimensiones = pd.concat(all_dimensiones).drop_duplicates().reset_index(drop=True)
    combined_dimensiones.to_csv(os.path.join(output_folder, 'dimensiones.csv'), index=False)

    # Combinar y guardar datos
    combined_datos = pd.concat(all_datos).reset_index(drop=True)
    combined_datos.to_csv(os.path.join(output_folder, 'datos.csv'), index=False)

    print("Todos los archivos han sido procesados y guardados.")

def verify_id_counts(datos_file):
    """
    Verifica cuántas filas tienen el mismo ID en el archivo de datos.
    """
    datos = pd.read_csv(datos_file)
    id_counts = datos['id'].value_counts()
    print("Conteo de filas por ID:")
    print(id_counts)
    return id_counts

# Procesar todos los CSV en la carpeta 'entrada' y guardar en la carpeta 'salida'
process_multiple_csv('csv_bien', 'prueba')

# Verificar los IDs en el archivo 'datos.csv'
verify_id_counts('prueba/datos.csv')


In [ ]:
# Extraer la primera fila del DataFrame
row = df.iloc[0].to_dict()
print("\nPrimera fila del DataFrame:", row)

# Función fetch_existing_dimensions (probamos si funciona correctamente)
def fetch_existing_dimensions():
    print("\n--- Ejecutando fetch_existing_dimensions ---") 
    response = supabase.table('tabladimension').select('*').execute()
    print("Response data:", response.data)

    if not response.data or len(response.data) == 0:
        # Si no hay datos en la tabla, definimos las columnas manualmente
        dimension_columns = ['Seccion', 'Capitulo', 'Partida', 'Arancel', 'Codigo_Seccion', 
                             'Codigo_Capitulo', 'Codigo_Partida', 'Codigo_Arancel', 
                             'Pais_Origen', 'Region', 'Via_Transporte', 'Colecturia_ID', 
                             'Colecturia', 'Regimen_Aduanero', 'Codigo_ISO', 'Categoria', 
                             'Frecuencia', 'Pais', 'Departamento', 'Sub-Categoria', 
                             'Unidad', 'Tipo', 'Ajuste', 'Fuente']
        return {}, dimension_columns

    columns = response.data[0].keys()
    dimension_columns = [col for col in columns if col != 'id']

    existing_dimensions = {}
    for r in response.data:
        dimension_combination = tuple(r[c] for c in dimension_columns)
        existing_dimensions[dimension_combination] = r['id']

    print("Existing dimensions:", existing_dimensions)
    print("Dimension columns:", dimension_columns)
    return existing_dimensions, dimension_columns

# Función create_dimension_combination
def create_dimension_combination(row):
    print("\n--- Ejecutando create_dimension_combination ---")
    combination = tuple(row[col] for col in dimension_columns)
    print("Dimension combination:", combination)
    return combination

# Función get_or_add_dimension
def get_or_add_dimension(dimension_combination):
    print("\n--- Ejecutando get_or_add_dimension ---")
    if dimension_combination in existing_dimensions:
        print("ID existente encontrado:", existing_dimensions[dimension_combination])
        return existing_dimensions[dimension_combination]
    else:
        new_dimension_data = dict(zip(dimension_columns, dimension_combination))
        print("Insertando nueva dimensión:", new_dimension_data)
        response = supabase.table('tabladimension').insert(new_dimension_data).execute()
        new_id = response.data[0]['id']
        print("Nuevo ID retornado:", new_id)
        existing_dimensions[dimension_combination] = new_id
        return new_id

# Proceso paso a paso con la primera fila
print("\n--- INICIANDO PRUEBA PASO A PASO ---")

# Paso 1: Fetch existing dimensions
existing_dimensions, dimension_columns = fetch_existing_dimensions()

# Paso 2: Crear la combinación de dimensiones
dimension_combination = create_dimension_combination(row)

# Paso 3: Obtener o agregar la dimensión
dimension_id = get_or_add_dimension(dimension_combination)

# Paso 4: Preparar datos para upsert
row['id'] = dimension_id
columns_to_keep = ['Fecha', 'Valor', 'id']
chunk_data_filtered = {k: row[k] for k in columns_to_keep}
print("\nDatos preparados para upsert:", chunk_data_filtered)

# Paso 5: Realizar el upsert en Supabase
try:
    print("\n--- Ejecutando upsert en datos_light ---")
    response = supabase.table('datos_light').upsert([chunk_data_filtered]).execute()
    print("Upsert response:", response.data)
except Exception as e:
    print("Error durante el upsert:", e)


In [ ]:
import pandas as pd
import os

# Función para normalizar un chunk de datos
def normalize_chunk(chunk, dimension_columns):
    """
    Normaliza las columnas relevantes en un chunk.
    """
    chunk[dimension_columns] = chunk[dimension_columns].apply(lambda x: x.astype(str).str.strip().str.lower())
    return chunk

# Función para procesar un chunk de datos
def process_chunk(chunk, dimension_columns):
    """
    Procesa un chunk, extrae dimensiones únicas y mapea IDs.
    """
    dimensiones = chunk[dimension_columns].drop_duplicates()
    dimensiones['id'] = range(1, len(dimensiones) + 1)
    datos = chunk.merge(dimensiones, on=dimension_columns, how='left')[['Fecha', 'Valor', 'id']]
    return dimensiones, datos

# Función para procesar un archivo CSV en chunks
def process_single_csv(file_path, output_folder, chunk_size=100000):
    """
    Procesa un archivo CSV en chunks y guarda los resultados parciales.
    """
    print(f"Procesando archivo: {file_path}")

    dimensiones_file = os.path.join(output_folder, 'dimensiones_temp.csv')
    datos_file = os.path.join(output_folder, 'datos_temp.csv')

    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
        # Modificar la columna 'Codigo_ISO' a 'DOM'
        if 'Codigo_ISO' in chunk.columns:
            chunk['Codigo_ISO'] = 'DOM'

        # Añadir la columna 'Ciudad'
        chunk['Ciudad'] = "Régimen de aduanas"

        # Eliminar valores nulos y normalizar
        chunk = chunk.dropna(subset=['Fecha', 'Valor']).drop_duplicates()
        dimension_columns = [col for col in chunk.columns if col not in ['Fecha', 'Valor']]
        chunk = normalize_chunk(chunk, dimension_columns)

        # Procesar el chunk
        dimensiones, datos = process_chunk(chunk, dimension_columns)

        # Guardar resultados parciales
        if not os.path.exists(dimensiones_file):
            dimensiones.to_csv(dimensiones_file, index=False)
        else:
            dimensiones.to_csv(dimensiones_file, mode='a', header=False, index=False)

        if not os.path.exists(datos_file):
            datos.to_csv(datos_file, index=False)
        else:
            datos.to_csv(datos_file, mode='a', header=False, index=False)

# Función para consolidar resultados parciales
def consolidate_results(output_folder):
    """
    Consolida los resultados parciales y genera los archivos finales.
    """
    dimensiones_file = os.path.join(output_folder, 'dimensiones_temp.csv')
    datos_file = os.path.join(output_folder, 'datos_temp.csv')

    if os.path.exists(dimensiones_file):
        dimensiones = pd.read_csv(dimensiones_file).drop_duplicates().reset_index(drop=True)
        dimensiones.to_csv(os.path.join(output_folder, 'dimensiones.csv'), index=False)
        os.remove(dimensiones_file)

    if os.path.exists(datos_file):
        datos = pd.read_csv(datos_file).reset_index(drop=True)
        datos.to_csv(os.path.join(output_folder, 'datos.csv'), index=False)
        os.remove(datos_file)

# Función para procesar múltiples archivos CSV
def process_multiple_csv(input_folder, output_folder, chunk_size=100000):
    """
    Procesa todos los archivos CSV en una carpeta.
    """
    os.makedirs(output_folder, exist_ok=True)
    archivos = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    for archivo in archivos:
        file_path = os.path.join(input_folder, archivo)
        process_single_csv(file_path, output_folder, chunk_size)

    consolidate_results(output_folder)

# Ejecutar el proceso
process_multiple_csv('csv_bien', 'prueba')


In [ ]:
df=pd.read_csv('prueba/dimensiones.csv')
len(df)

## SUBIDA

In [1]:
import pandas as pd

df_final=pd.read_csv('completo_inicial.csv', encoding='utf-8')

In [2]:
# Crear una copia del DataFrame original
working_df = df_final.copy()

In [ ]:
import pandas as pd

# =============================================================================
# 1. CONFIGURACIÓN INICIAL
# =============================================================================
# Este bloque establece columnas iniciales con valores estáticos y ordena el DataFrame.
# Se asume que 'working_df' ya está definido anteriormente.

# Añadir columnas iniciales con valores estáticos
working_df['Código ISO'] = 'DOM'
working_df['País'] = 'República Dominicana'
working_df['Departamento'] = 'Nacional'
working_df['Ciudad'] = 'Nacional'
working_df['Categoría'] = 'Sector Externo'
working_df['Sub-Categoría'] = 'Importaciones'

# Ordenar las columnas en el DataFrame (ajusta si alguna no existe en tu DF)
column_order = [
    'Código ISO', 'País', 'Departamento', 'Ciudad', 'Categoría', 'Sub-Categoría',
    'Pais_Origen', 'Region', 'Via_Transporte', 'Colecturia', 'Regimen_Aduanero',
    'Cod_Seccion', 'Cod_Capitulo', 'Cod_Partida', 'Cod_Arancel',
    'Transacción', 'Año', 'Mes', 'Desc_Seccion', 'Desc_Capitulo',
    'Desc_Partida', 'Desc_Arancel', 'Peso', 'V_FOB_USD', 'V_CIF_USD',
    'Colecturia_ID'
]
working_df = working_df[column_order]

# =============================================================================
# 2. PREPROCESAMIENTO DE DATOS
# =============================================================================
# Limpiar guiones "-" en las columnas descriptivas y generar métricas base.
# También se crea la columna 'Fecha' y se eliminan 'Año'/'Mes'.

for col in ['Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel']:
    working_df[col] = working_df[col].str.replace('-', ' ').str.strip()

# Crear columnas derivadas
working_df['Toneladas'] = working_df['Peso'] / 1000.0
working_df['Millones_USD_FOB'] = working_df['V_FOB_USD'] / 1_000_000
working_df['Millones_Toneladas_metricas'] = working_df['Toneladas'] / 1_000_000
working_df['Millones_USD_CIF'] = working_df['V_CIF_USD'] / 1_000_000

# Crear columna 'Fecha' a partir de 'Año' y 'Mes'
working_df['Fecha'] = pd.to_datetime(
    working_df['Año'].astype(str) + '-' + working_df['Mes'].apply(lambda x: f'{x:02}') + '-01'
)
working_df.drop(['Año', 'Mes'], axis=1, inplace=True)

# Ordenar por criterios lógicos
working_df.sort_values(
    by=['Fecha', 'Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel'],
    ascending=True,
    inplace=True
)

# =============================================================================
# 3. AGRUPACIÓN MENSUAL Y SUMA MÓVIL 12 MESES
# =============================================================================
agrupadas = working_df.groupby(
    [
        'Fecha', 'Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel',
        'Cod_Seccion', 'Cod_Capitulo', 'Cod_Partida', 'Cod_Arancel'
    ],
    as_index=False
).agg({
    'Toneladas': 'sum',
    'Millones_USD_FOB': 'sum',
    'Millones_Toneladas_metricas': 'sum',
    'Millones_USD_CIF': 'sum',
    'Pais_Origen': 'first',
    'Region': 'first',
    'Via_Transporte': 'first',
    'Colecturia_ID': 'first',
    'Colecturia': 'first',
    'Regimen_Aduanero': 'first'
})

# Extraer mes para rolling y variación
agrupadas['Mes'] = agrupadas['Fecha'].dt.month

# Rolling 12M
grouped = agrupadas.groupby(['Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida'])
agrupadas['Toneladas_12M'] = grouped['Toneladas'].transform(lambda x: x.rolling(12, min_periods=1).sum())
agrupadas['Millones_USD_FOB_12M'] = grouped['Millones_USD_FOB'].transform(lambda x: x.rolling(12, min_periods=1).sum())
agrupadas['Millones_Toneladas_metricas_12M'] = grouped['Millones_Toneladas_metricas'].transform(lambda x: x.rolling(12, min_periods=1).sum())
agrupadas['Millones_USD_CIF_12M'] = grouped['Millones_USD_CIF'].transform(lambda x: x.rolling(12, min_periods=1).sum())

# =============================================================================
# 4. VALOR ANTERIOR (AÑO PREVIO) Y VARIACIÓN
# =============================================================================
columnas_id = [
    'Fecha', 'Desc_Seccion', 'Desc_Capitulo', 'Desc_Partida', 'Desc_Arancel',
    'Cod_Seccion', 'Cod_Capitulo', 'Cod_Partida', 'Cod_Arancel', 'Mes',
    'Pais_Origen', 'Region', 'Via_Transporte', 'Colecturia_ID', 'Colecturia',
    'Regimen_Aduanero'
]
columnas_con_valor = [c for c in agrupadas.columns if c not in columnas_id]

for col in columnas_con_valor:
    agrupadas[f'{col}_valor_anterior'] = agrupadas.groupby('Mes')[col].shift(12)
    agrupadas[f'variacion {col}'] = (
        (agrupadas[col] - agrupadas[f'{col}_valor_anterior']) /
        agrupadas[f'{col}_valor_anterior'] * 100
    )

# Excluir columnas *_valor_anterior antes de melt
final_columnas_con_valor = [
    c for c in agrupadas.columns
    if c not in columnas_id and 'valor_anterior' not in c.lower()
]

# =============================================================================
# 5. TRANSFORMACIÓN A LARGO (MELT) CON CHUNKING POR FILAS
# =============================================================================
chunksize = 50_000
num_rows = len(agrupadas)
df_long_list = []

for start in range(0, num_rows, chunksize):
    end = start + chunksize
    subset = agrupadas.iloc[start:end].copy()

    df_partial = pd.melt(
        subset,
        id_vars=columnas_id,
        value_vars=final_columnas_con_valor,
        value_name='Valor',            # <--- AQUÍ se crea la columna 'Valor'
        var_name='Columna_desagregacion'
    )
    df_long_list.append(df_partial)
    del df_partial
    del subset

df_long = pd.concat(df_long_list, ignore_index=True)
del df_long_list

# =============================================================================
# 6. AÑADIR COLUMNAS EXTRAS (CON INFORMACIÓN FIJA) Y FUNCIONES
# =============================================================================
df_long = df_long.assign(
    Codigo_ISO='DOM',
    Categoria='Sector Externo',
    Frecuencia='Mensual',
    Fuente='ONE',
    Pais='República Dominicana',
    Departamento='Nacional',
    Ciudad='Nacional',
    Sub_Categoria='Importaciones'
)

# Funciones para Unidad, Tipo, Ajuste
def columna_unidad(row):
    var = str(row['Columna_desagregacion']).lower()
    if 'porcentaje_importaciones_totales' in var or 'participacion_importaciones_totales' in var:
        return '% de las Importaciones totales'
    elif 'usd_por_ton' in var or 'precio_implicito' in var:
        return 'USD por tonelada'
    elif 'variacion' in var:
        return '% variación anual'
    elif 'millones' in var:
        return 'Millones'
    return 'Unidad desconocida'

def columna_tipo(row):
    var = str(row['Columna_desagregacion']).lower()
    if 'fob' in var:
        return 'USD FOB'
    elif 'cif' in var:
        return 'USD CIF'
    elif 'toneladas' in var and 'variacion' not in var:
        return 'Toneladas métricas'
    elif 'variacion' in var:
        return 'Precio implícito'
    return 'Tipo desconocido'

def columna_ajuste(row):
    var = str(row['Columna_desagregacion']).lower()
    if '12m' in var:
        return 'Suma móvil 12 meses'
    return 'Serie mensual'

df_long['Unidad'] = df_long.apply(columna_unidad, axis=1)
df_long['Tipo'] = df_long.apply(columna_tipo, axis=1)
df_long['Ajuste'] = df_long.apply(columna_ajuste, axis=1)

# =============================================================================
# 7. RENOMBRAR COLUMNAS AL FORMATO DEFINITIVO
# =============================================================================

# Creamos un diccionario de renombrado para que tus columnas finales
# queden en los nombres requeridos:
rename_dict = {
    # Desagregaciones colapsadas (después del melt):
    'Desagregacion-1': 'Seccion',
    'Desagregacion-2': 'Capitulo',
    'Desagregacion-3': 'Partida',
    'Desagregacion-4': 'Arancel',

    # Códigos
    'Cod_Seccion': 'Codigo_Seccion',
    'Cod_Arancel': 'Codigo_Arancel',
    'Cod_Capitulo': 'Codigo_Capitulo',
    'Cod_Partida': 'Codigo_Partida',

    # Subcategoría
    'Sub_Categoria': 'Sub-Categoria',

    # Regimen
    'Regimen_Aduanero': 'Regimen_Adicional'
}

df_long.rename(columns=rename_dict, inplace=True)

# =============================================================================
# 8. REORDENAR COLUMNAS SEGÚN LISTA SOLICITADA
# =============================================================================
ordered_columns = [
    'Fecha',
    'Valor',
    'Sub-Categoria',
    'Tipo',
    'Seccion',
    'Unidad',
    'Frecuencia',
    'Ajuste',
    'Fuente',
    'Pais',
    'Departamento',
    'Codigo_ISO',
    'Categoria',
    'Codigo_Seccion',
    'Codigo_Arancel',
    'Codigo_Capitulo',
    'Codigo_Partida',
    'Capitulo',         # vendrá de 'Desagregacion-2' si existía
    'Arancel',          # vendrá de 'Desagregacion-4'
    'Partida',          # vendrá de 'Desagregacion-3'
    'Region',
    'Pais_Origen',
    'Via_Transporte',
    'Regimen_Adicional',
    'Colecturia_ID',
    'Colecturia',
    'Ciudad'
]

# Alinear el DataFrame a este orden (solo columnas que existan)
cols_existentes = [c for c in ordered_columns if c in df_long.columns]
df_long = df_long.reindex(columns=cols_existentes + [c for c in df_long.columns if c not in cols_existentes])

# =============================================================================
# 9. RESULTADO FINAL
# =============================================================================
print(df_long.head(10))
print("Columnas finales:", df_long.columns.tolist())



In [ ]:
df_long

In [ ]:
# Lista de columnas nuevas en orden
ordered_columns = [
    'Fecha',
    'Valor',
    'Sub-Categoria',
    'Tipo',
    'Seccion',
    'Unidad',
    'Frecuencia',
    'Ajuste',
    'Fuente',
    'Pais',
    'Departamento',
    'Codigo_ISO',
    'Categoria',
    'Codigo_Seccion',
    'Codigo_Arancel',
    'Codigo_Capitulo',
    'Codigo_Partida',
    'Capitulo',  # Vendrá de 'Desc_Capitulo' (si existía)
    'Arancel',   # Vendrá de 'Desc_Arancel'
    'Partida',   # Vendrá de 'Desc_Partida'
    'Region',
    'Pais_Origen',
    'Via_Transporte',
    'Regimen_Aduanero',  # Vendrá de 'Regimen_Adicional'
    'Colecturia_ID',
    'Colecturia',
]

# Mapeo para renombrar columnas
column_mapping = {
    'Desc_Seccion':'Seccion',
    'Desc_Capitulo': 'Capitulo',
    'Desc_Arancel': 'Arancel',
    'Desc_Partida': 'Partida',
    'Regimen_Adicional': 'Regimen_Aduanero'
}

# Renombrar las columnas del DataFrame según el mapeo
df_long.rename(columns=column_mapping, inplace=True)

# Reordenar columnas según la lista 'ordered_columns'
df_long = df_long[ordered_columns]

# Mostrar el DataFrame resultante
print(df_long.head())


In [ ]:
df_long.dtypes

In [ ]:
# Filas con al menos un valor NaN o None
filas_nan = df_long[df_long.isnull().any(axis=1)]

# Mostrar las filas encontradas
print(filas_nan)
df_long.dropna(inplace=True)
filas_nan = df_long[df_long.isnull().any(axis=1)]
print(filas_nan)

In [ ]:
# Supongdescribe(columns={mos que tu DataFrame se llama df y la columna que deseas ver es 'nombre_columna'

# Mostrar la columna como una Serie de pandas
print(df_long['Valor'])

# Si prefieres verla como un DataFrame de una sola columna
print(df_long[['Valor']])


In [4]:
df_long.drop(['Mes', 'Ciudad','Columna_desagregacion'], axis=1, inplace=True)


In [ ]:
import duckdb
import pandas as pd

# 1) Conexión a un archivo local DuckDB (se crea si no existe)
con = duckdb.connect("mi_bd_local.duckdb")

#------------------------------------------------------------------------------
# EJEMPLO DE DataFrame con columna "Sub-Categoria" y otras
#------------------------------------------------------------------------------



# 2) Registrar df_long en DuckDB => Tabla big_table
con.register("df_long", df_long)
con.execute("CREATE OR REPLACE TABLE big_table AS SELECT * FROM df_long")
print("Tabla big_table creada en DuckDB.")

# Definir columnas de dimensión (todas menos 'Fecha' y 'Valor')
dimension_cols = [c for c in df_long.columns if c not in ['Fecha', 'Valor']]

# Si alguna columna tiene espacios o guiones, ponerla entre comillas para SQL
dimension_cols_quoted = [f'"{col}"' for col in dimension_cols]
cols_str = ", ".join(dimension_cols_quoted)

# 3) Crear dimension_table con las columnas de dimensión (DISTINCT)
create_dim_sql = f"""
    CREATE OR REPLACE TABLE dimension_table AS
    SELECT DISTINCT {cols_str}
    FROM big_table
"""
con.execute(create_dim_sql)
print("Tabla dimension_table creada con las columnas de dimensión (DISTINCT).")

# 4) Generar dimension_id con row_number() sin usar UPDATE
#    Creamos una tabla nueva (dimension_table_new) y luego reemplazamos.
assign_id_sql = f"""
    CREATE OR REPLACE TABLE dimension_table_new AS
    SELECT
        row_number() OVER (ORDER BY {cols_str}) AS dimension_id,
        d.*
    FROM dimension_table d
"""
con.execute(assign_id_sql)

con.execute("DROP TABLE dimension_table")
con.execute("ALTER TABLE dimension_table_new RENAME TO dimension_table")
print("Agregada columna dimension_id usando row_number().")

# 5) Crear la fact_table con JOIN: dimension_id, Fecha, Valor
on_clause = " AND ".join([f'b."{col}" = d."{col}"' for col in dimension_cols])
create_fact_sql = f"""
    CREATE OR REPLACE TABLE fact_table AS
    SELECT 
        d.dimension_id,
        b."Fecha" AS Fecha,
        b."Valor" AS Valor
    FROM big_table b
    JOIN dimension_table d
      ON {on_clause}
"""
con.execute(create_fact_sql)
print("Tabla fact_table creada (JOIN).")

# 6) (Opcional) Verificar resultados
dim_count = con.execute("SELECT COUNT(*) FROM dimension_table").fetchone()[0]
fact_count = con.execute("SELECT COUNT(*) FROM fact_table").fetchone()[0]
print("dimension_table registros:", dim_count)
print("fact_table registros:", fact_count)

# 7) (Opcional) Exportar a parquet
con.execute("""COPY dimension_table TO 'dimension_table.parquet' (FORMAT 'parquet')""")
con.execute("""COPY fact_table TO 'fact_table.parquet' (FORMAT 'parquet')""")
print("Exportadas dimension_table.parquet y fact_table.parquet.")



In [10]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():
    secret_name = "RepDom-DB"
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e

    secret = json.loads(get_secret_value_response['SecretString'])
    return secret

secretos = get_secret()
url_supabase: str = secretos['supabase_URL_RD']
key: str = secretos['supabase_key_RD']

# Instala la librería supabase-python si no la tienes: pip install supabase
from supabase import create_client, Client

supabase: Client = create_client(url_supabase, key)


In [14]:
def show_rows_with_dimension_id(con, table_name: str, dim_id: int, limit=5):
    """
    Muestra hasta 'limit' filas de 'table_name' donde dimension_id = dim_id.
    """
    query = f"""
        SELECT *
        FROM {table_name}
        WHERE dimension_id = {dim_id}
        LIMIT {limit}
    """
    df = con.execute(query).fetchdf()
    print(f"===== HEAD de la tabla {table_name} con dimension_id = {dim_id} (máx {limit} filas) =====")
    print(df)


In [ ]:
# Ejemplo
# Muestra 5 filas de 'fact_table' que tengan dimension_id = 100
show_rows_with_dimension_id(con, 'fact_table', 1, 5)

# Muestra 5 filas de 'dimension_table' con dimension_id = 100
show_rows_with_dimension_id(con, 'dimension_table', 1, 5)


In [ ]:
import duckdb
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import json
from supabase import create_client, Client

def get_secret():
    secret_name = "RepDom-DB"
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e

    secret = json.loads(get_secret_value_response['SecretString'])
    return secret


def upload_same_dimension_id(
    con: duckdb.DuckDBPyConnection,
    supabase_client: Client,
    dimension_table_name: str = "dimension_table",
    fact_table_name: str = "fact_table",
    dimension_target: str = "tabladimension",
    fact_target: str = "datos_light",
    same_dimension_id: int = 1
):
    """
    Este método:
      1) Selecciona todas las filas de `dimension_table` en DuckDB donde dimension_id = same_dimension_id
      2) Selecciona todas las filas de `fact_table` donde dimension_id = same_dimension_id
      3) Renombra 'dimension_id' -> 'id' en ambos DataFrames (asumiendo que
         en las tablas de Supabase la columna de PK/FK se llama 'id').
      4) Convierte la columna 'Fecha' en la fact_table a string (YYYY-MM-DD), 
         para evitar error de JSON serializable.
      5) Hace upsert de todas las filas de la dimensión en `dimension_target`.
      6) Hace upsert de todas las filas de la fact table en `fact_target`.
    
    De esta forma, en Supabase tendrás:
      - `tabladimension` con 'id' = same_dimension_id
      - `datos_light` con 'id' = same_dimension_id
    y así las dos tablas comparten el mismo ID.
    
    Ajusta según tu modelo real:
      - Si en `datos_light` la columna se llama 'dimension_id', 
        no renombres y crea esa columna en Supabase.
    """

    # 1) Leer las filas de dimension_table con dimension_id = same_dimension_id
    query_dim = f"""
        SELECT *
        FROM {dimension_table_name}
        WHERE dimension_id = {same_dimension_id}
    """
    df_dim = con.execute(query_dim).fetchdf()
    if df_dim.empty:
        print(f"No se encontraron filas en {dimension_table_name} con dimension_id={same_dimension_id}")
    else:
        print(f"Filas de {dimension_table_name} con dimension_id={same_dimension_id}: {len(df_dim)}")
        # Renombrar 'dimension_id' -> 'id'
        if 'dimension_id' in df_dim.columns:
            df_dim.rename(columns={'dimension_id': 'id'}, inplace=True)

    # 2) Leer las filas de fact_table con dimension_id = same_dimension_id
    query_fact = f"""
        SELECT *
        FROM {fact_table_name}
        WHERE dimension_id = {same_dimension_id}
    """
    df_fact = con.execute(query_fact).fetchdf()
    if df_fact.empty:
        print(f"No se encontraron filas en {fact_table_name} con dimension_id={same_dimension_id}")
    else:
        print(f"Filas de {fact_table_name} con dimension_id={same_dimension_id}: {len(df_fact)}")

        # Convertir la columna 'Fecha' a string para JSON
        if 'Fecha' in df_fact.columns and pd.api.types.is_datetime64_any_dtype(df_fact['Fecha']):
            df_fact['Fecha'] = df_fact['Fecha'].dt.strftime('%Y-%m-%d')

        # Renombrar 'dimension_id' -> 'id'
        if 'dimension_id' in df_fact.columns:
            df_fact.rename(columns={'dimension_id': 'id'}, inplace=True)

    # 3) Subir la dimensión
    if not df_dim.empty:
        dimension_dicts = df_dim.to_dict(orient='records')
        try:
            response_dim = supabase_client.table(dimension_target).upsert(dimension_dicts).execute()
            print(f"Upsert dimensión completado. Respuesta:\n{response_dim}")
        except Exception as e:
            print("Error al subir dimensión a Supabase:", e)

    # 4) Subir la fact table
    if not df_fact.empty:
        fact_dicts = df_fact.to_dict(orient='records')
        try:
            response_fact = supabase_client.table(fact_target).upsert(fact_dicts).execute()
            print(f"Upsert fact completado. Respuesta:\n{response_fact}")
        except Exception as e:
            print("Error al subir fact a Supabase:", e)


# Ejemplo de uso principal
if __name__ == "__main__":
    # 1) Obtener credenciales
    secretos = get_secret()
    url_supabase = secretos['supabase_URL_RD']
    key = secretos['supabase_key_RD']

    # 2) Crear cliente Supabase
    supabase_client = create_client(url_supabase, key)

    # 3) Conexión DuckDB
    con = duckdb.connect("mi_bd_local.duckdb")

    # 4) Subir TODAS las filas que tengan dimension_id = 1
    #    (o el dimension_id que necesites) a Supabase
    upload_same_dimension_id(
        con=con,
        supabase_client=supabase_client,
        dimension_table_name="dimension_table",
        fact_table_name="fact_table",
        dimension_target="tabladimension",
        fact_target="datos_light",
        same_dimension_id=1  # Ajusta el ID que deseas
    )


